In [68]:
import os
import torch
import numpy as np
import glob
import albumentations
from PIL import Image
from torch.utils import data
from torch import nn
import torchvision.models as models

In [6]:
DATA_PATH = '/home/jupyter/datascience/chest-xray-pneumonia/data/chest_xray'
! pip install -q kaggle
from google.colab import files
files.upload()

IndentationError: ignored

In [8]:
 ! mkdir ~/.kaggle 

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [10]:
 ! cp kaggle.json ~/.kaggle/
 ! ls ~/.kaggle/

kaggle.json


In [12]:
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:27<00:00, 102MB/s] 
100% 2.29G/2.29G [00:27<00:00, 88.0MB/s]


In [13]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! ls /content
! unzip chest-xray-pneumonia.zip 

In [63]:
class ClassificationDataset:
    def __init__(self,image_paths,targets,resize=None):
        self.image_paths=image_paths
        self.targets = targets
        self.resize = resize
        self.aug= albumentations.Compose([
            albumentations.Rotate(),
            albumentations.Normalize(always_apply=True)
        ])
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self,item):
        path = self.image_paths[item]
        image = Image.open(path).convert("RGB")
        target = self.targets[item]
        if self.resize is not None:
            image =image.resize((self.resize[1],self.resize[0]),resample=Image.BILINEAR)
        
        image = np.array(image)
        augmented = self.aug(image=image)
        image = augmented["image"]
        image = np.transpose(image,(2,0,1)).astype(np.float32)
        image = torch.tensor(data=image,dtype=torch.float)
        return image,

In [27]:
! ls /content/chest_xray/train

NORMAL	PNEUMONIA


In [62]:
DATA_PATH
train_images = glob.glob("/content/chest_xray/train/**/**.jpeg")
train_targets = [(0 if (x.split("/")[-2]=='PNEUMONIA') else 1 ) for x in train_images]

ds = ClassificationDataset(train_images,train_targets,resize=(24,24))


('/content/chest_xray/train/PNEUMONIA/person1318_virus_2274.jpeg', 0)

In [66]:
training_data=data.DataLoader(dataset=ds,batch_size=64)

In [69]:
learner=models.resnet34(pretrained=True,progress=True)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth


In [70]:
learner(ds)

TypeError: ignored